# Integrantes

Gabriel Luni Nakashima RM558096

Gabriel Lacerda Araujo RM558307

# IMPORTS

In [1]:
import random
import numpy as np
import itertools
import math
import time

## Dados da habilidades (GRAFO)

In [2]:
# --- Base de dados das habilidades (corrigida) ---

habilidades = {
    'S1': {'Nome': 'Programação Básica (Python)', 'Tempo': 80, 'Valor': 3, 'Complexidade': 4, 'Pre_Reqs': []},
    'S2': {'Nome': 'Modelagem de Dados (SQL)', 'Tempo': 60, 'Valor': 4, 'Complexidade': 3, 'Pre_Reqs': []},
    'S3': {'Nome': 'Algoritmos Avançados', 'Tempo': 100, 'Valor': 7, 'Complexidade': 8, 'Pre_Reqs': ['S1']},
    'S4': {'Nome': 'Fundamentos de Machine Learning', 'Tempo': 120, 'Valor': 8, 'Complexidade': 9, 'Pre_Reqs': ['S1', 'S3']},
    'S5': {'Nome': 'Visualização de Dados (BI)', 'Tempo': 40, 'Valor': 6, 'Complexidade': 5, 'Pre_Reqs': ['S2']},
    'S6': {'Nome': 'IA Generativa Ética', 'Tempo': 150, 'Valor': 10, 'Complexidade': 10, 'Pre_Reqs': ['S4']},
    'S7': {'Nome': 'Estruturas em Nuvem (AWS/Azure)', 'Tempo': 70, 'Valor': 5, 'Complexidade': 7, 'Pre_Reqs': []},
    'S8': {'Nome': 'APIs e Microsserviços', 'Tempo': 90, 'Valor': 6, 'Complexidade': 6, 'Pre_Reqs': ['S1']},
    'S9': {'Nome': 'DevOps & CI/CD', 'Tempo': 110, 'Valor': 9, 'Complexidade': 8, 'Pre_Reqs': ['S7', 'S8']},
    'H10': {'Nome': 'Segurança de Dados', 'Tempo': 60, 'Valor': 5, 'Complexidade': 6, 'Pre_Reqs': []},
    'H11': {'Nome': 'Análise de Big Data', 'Tempo': 90, 'Valor': 8, 'Complexidade': 8, 'Pre_Reqs': ['S4']},
    'H12': {'Nome': 'Introdução a IoT', 'Tempo': 30, 'Valor': 3, 'Complexidade': 3, 'Pre_Reqs': []}
}


# Desafio 1

### Funções de verificação

In [3]:
def detectar_ciclo():
    visitados, pilha = set(), set()
    def dfs(n):
        if n in pilha:
            return True
        if n in visitados:
            return False
        visitados.add(n)
        pilha.add(n)
        for prereq in habilidades[n]['Pre_Reqs']:
            if dfs(prereq):
                return True
        pilha.remove(n)
        return False
    return any(dfs(n) for n in habilidades)

def encontrar_nos_orfaos():
    dependentes = {d for v in habilidades.values() for d in v['Pre_Reqs']}
    return [n for n in habilidades if not habilidades[n]['Pre_Reqs']]



### Otimização de caminho

In [4]:
melhor_valor = 0
melhor_caminho = []
tempo_limite = 350
complexidade_limite = 30

# --- Função para encontrar conexões válidas (arestas direcionadas) ---
def get_vizinhos(atual):
    vizinhos = []
    for prox in habilidades:
        if atual in habilidades[prox]['Pre_Reqs']:
            vizinhos.append(prox)
    return vizinhos

# --- Busca recursiva pelos caminhos válidos ---
def buscar_caminhos(atual, caminho, tempo, comp, valor):
    global melhor_valor, melhor_caminho
    if tempo > tempo_limite or comp > complexidade_limite:
        return
    if atual == 'S6':  # chegou no objetivo final
        if valor > melhor_valor:
            melhor_valor = valor
            melhor_caminho = caminho[:]
        return

    for prox in get_vizinhos(atual):
        if prox not in caminho:  # evita ciclos
            buscar_caminhos(
                prox,
                caminho + [prox],
                tempo + habilidades[prox]['Tempo'],
                comp + habilidades[prox]['Complexidade'],
                valor + habilidades[prox]['Valor']
            )


### Execução

In [5]:
melhor_valor = 0
melhor_caminho = []

if detectar_ciclo():
    print("Erro: o grafo possui ciclos!")
else:
    orfaos = encontrar_nos_orfaos()
    print(f"Nós iniciais (sem pré-requisitos): {orfaos}")

    for inicio in orfaos:
        buscar_caminhos(inicio, [inicio],
                        habilidades[inicio]['Tempo'],
                        habilidades[inicio]['Complexidade'],
                        habilidades[inicio]['Valor'])

    print("\n--- MELHOR CAMINHO ENCONTRADO ---")
    if melhor_caminho:
        for s in melhor_caminho:
            print(f"{s} -> {habilidades[s]['Nome']}")
        print(f"\nValor total: {melhor_valor}")
        print(f"Tempo total: {sum(habilidades[s]['Tempo'] for s in melhor_caminho)}h")
        print(f"Complexidade total: {sum(habilidades[s]['Complexidade'] for s in melhor_caminho)}")
    else:
        print("Nenhum caminho válido até S6 foi encontrado dentro das restrições.")


Nós iniciais (sem pré-requisitos): ['S1', 'S2', 'S7', 'H10', 'H12']

--- MELHOR CAMINHO ENCONTRADO ---
S1 -> Programação Básica (Python)
S4 -> Fundamentos de Machine Learning
S6 -> IA Generativa Ética

Valor total: 21
Tempo total: 350h
Complexidade total: 23


# Desafio 2

### Validação do Grafo

In [6]:
# --- Validação do Grafo (corrigido com 'Pre_Reqs') ---

def validar_grafo(habilidades):
    erros = []

    def dfs(no, visitados, pilha):
        visitados.add(no)
        pilha.add(no)
        for pre in habilidades[no]['Pre_Reqs']:
            if pre not in habilidades:
                erros.append(f"Erro: '{pre}' não existe mas é pré-requisito de '{no}'.")
            elif pre not in visitados:
                dfs(pre, visitados, pilha)
            elif pre in pilha:
                erros.append(f"Ciclo detectado envolvendo '{no}' e '{pre}'.")
        pilha.remove(no)

    visitados = set()
    for h in habilidades:
        if h not in visitados:
            dfs(h, visitados, set())

    if erros:
        print("\n Erros encontrados no grafo:")
        for e in erros:
            print(" -", e)
        return False
    else:
        print("Grafo validado: sem ciclos ou pré-requisitos inexistentes.")
        return True


### Cálculo de Permutações e Custos

In [7]:
# --- Cálculo das 120 Permutações e Custos ---

import itertools

def calcular_custo_ordem(ordem, habilidades):
    tempo_total = 0
    espera_total = 0
    concluidas = set()

    for h in ordem:
        for pre in habilidades[h]['Pre_Reqs']:
            if pre not in concluidas:
                espera_total += habilidades[pre]['Tempo']
        tempo_total += habilidades[h]['Tempo']
        concluidas.add(h)

    return tempo_total + espera_total

def analisar_permutacoes(habilidades):
    criticas = ['S3', 'S5', 'S7', 'S8', 'S9']
    resultados = []

    for ordem in itertools.permutations(criticas):
        custo = calcular_custo_ordem(ordem, habilidades)
        resultados.append((ordem, custo))

    resultados.sort(key=lambda x: x[1])
    return resultados


### Execução e Heurísticas

In [8]:
# --- Execução e análise das três melhores ordens ---

if validar_grafo(habilidades):
    resultados = analisar_permutacoes(habilidades)

    print("\n--- TOP 3 ORDENS MAIS EFICIENTES ---")
    for i, (ordem, custo) in enumerate(resultados[:3], start=1):
        print(f"{i}. Ordem: {' → '.join(ordem)} | Custo Total: {custo}")

    media_top3 = sum(c for _, c in resultados[:3]) / 3
    print(f"\nCusto médio das 3 melhores ordens: {media_top3:.2f}")

    print("\n Heurística observada:")
    print("- As ordens que iniciam com S3 (Algoritmos Avançados) reduzem espera, pois destravam S4 e S6.")
    print("- S7 (Nuvem) e S8 (APIs) geralmente aparecem antes de S9 (DevOps), pois são pré-requisitos dele.")
    print("- A ordem ideal prioriza dependências críticas e minimiza esperas em série.")


Grafo validado: sem ciclos ou pré-requisitos inexistentes.

--- TOP 3 ORDENS MAIS EFICIENTES ---
1. Ordem: S3 → S5 → S7 → S8 → S9 | Custo Total: 630
2. Ordem: S3 → S5 → S8 → S7 → S9 | Custo Total: 630
3. Ordem: S3 → S7 → S5 → S8 → S9 | Custo Total: 630

Custo médio das 3 melhores ordens: 630.00

 Heurística observada:
- As ordens que iniciam com S3 (Algoritmos Avançados) reduzem espera, pois destravam S4 e S6.
- S7 (Nuvem) e S8 (APIs) geralmente aparecem antes de S9 (DevOps), pois são pré-requisitos dele.
- A ordem ideal prioriza dependências críticas e minimiza esperas em série.


# Desafio 3

### Identificar habilidades básicas

In [9]:
# Identificar habilidades básicas (sem pré-requisitos)

# Assume que 'habilidades' já existe no ambiente (como nas células anteriores).
# Se não existir, redefina-o como no Desafio 1/2.

def habilidades_basicas(habs):
    return [hid for hid, meta in habs.items() if not meta.get('Pre_Reqs')]

basicas = habilidades_basicas(habilidades)
print("Habilidades básicas:", basicas)


Habilidades básicas: ['S1', 'S2', 'S7', 'H10', 'H12']


### Heurística gulosa (razão V/T)

In [10]:
# Algoritmo guloso: escolher por maior razão V/T até S >= 15

def greedy_v_per_t(basics, habs, alvo=15):
    disponiveis = basics[:]  # copiar
    selecionadas = []
    soma_v = 0
    soma_t = 0

    # calcular razão e ordenar dinamicamente (recomputar não necessário aqui porque dados fixos)
    # mas escolhemos iterativamente o melhor restante
    while soma_v < alvo and disponiveis:
        # escolher por maior V/T (em tie-break, escolher maior V)
        best = max(disponiveis, key=lambda h: (habs[h]['Valor'] / habs[h]['Tempo'], habs[h]['Valor']))
        disponiveis.remove(best)
        selecionadas.append(best)
        soma_v += habs[best]['Valor']
        soma_t += habs[best]['Tempo']

    return {
        'selecionadas': selecionadas,
        'valor': soma_v,
        'tempo': soma_t,
        'atingiu': soma_v >= alvo
    }

# Exemplo de uso (rodar após BLOCO 1)
res_greedy = greedy_v_per_t(basicas, habilidades, alvo=15)
print("GULOSO -> selecionadas:", res_greedy['selecionadas'])
print("Valor total:", res_greedy['valor'], "| Tempo total:", res_greedy['tempo'], "| Atingiu:", res_greedy['atingiu'])


GULOSO -> selecionadas: ['H12', 'H10', 'S7', 'S2']
Valor total: 17 | Tempo total: 220 | Atingiu: True


### Busca exaustiva sobre subconjuntos básicos

In [11]:
# Busca exaustiva (todas as combinações) para encontrar solução ótima mín. tempo com S >= 15

def exhaustive_optimal(basics, habs, alvo=15):
    melhor_sub = None
    melhor_tempo = math.inf
    melhor_val = 0

    # percorrer todos os subconjuntos não vazios
    for r in range(1, len(basics)+1):
        for comb in itertools.combinations(basics, r):
            soma_v = sum(habs[h]['Valor'] for h in comb)
            soma_t = sum(habs[h]['Tempo'] for h in comb)
            if soma_v >= alvo:
                if soma_t < melhor_tempo:
                    melhor_tempo = soma_t
                    melhor_sub = comb
                    melhor_val = soma_v
    if melhor_sub is None:
        return {'selecionadas': [], 'valor': 0, 'tempo': None, 'atingiu': False}
    return {'selecionadas': list(melhor_sub), 'valor': melhor_val, 'tempo': melhor_tempo, 'atingiu': True}

# Exemplo de uso (rodar após BLOCO 1)
res_opt = exhaustive_optimal(basicas, habilidades, alvo=15)
print("ÓTIMO -> selecionadas:", res_opt['selecionadas'])
print("Valor total:", res_opt['valor'], "| Tempo total:", res_opt['tempo'], "| Atingiu:", res_opt['atingiu'])


ÓTIMO -> selecionadas: ['S2', 'S7', 'H10', 'H12']
Valor total: 17 | Tempo total: 220 | Atingiu: True


### Comparação + Complexidade

In [12]:
# Comparação direta entre guloso e exaustivo, com saída resumida
res_greedy = greedy_v_per_t(basicas, habilidades, alvo=15)
res_opt = exhaustive_optimal(basicas, habilidades, alvo=15)

print("=== COMPARAÇÃO ===")
print("Guloso: selecionadas =", res_greedy['selecionadas'], "| Valor =", res_greedy['valor'], "| Tempo =", res_greedy['tempo'], "| Atingiu =", res_greedy['atingiu'])
print("Ótimo : selecionadas =", res_opt['selecionadas'],   "| Valor =", res_opt['valor'],   "| Tempo =", res_opt['tempo'],   "| Atingiu =", res_opt['atingiu'])

if res_greedy['atingiu'] and res_opt['atingiu']:
    if res_greedy['tempo'] == res_opt['tempo']:
        print("\nResultado: o guloso encontrou solução ótima (mesmo tempo).")
    elif res_greedy['tempo'] > res_opt['tempo']:
        print("\nResultado: guloso NÃO ótimo — solução ótima tem tempo menor.")
    else:
        print("\nResultado: guloso melhor que o exaustivo (estranho) — verifique implementações.")
else:
    print("\nAviso: uma das abordagens não atingiu o alvo S >= 15.")

print("\nComplexidade:")
print("- Guloso: O(n log n) se ordenar por razão no início (aqui O(n^2) em implementação simples), muito rápido para muitas habilidades.")
print("- Exaustivo: O(2^n) subconjuntos (n = número de habilidades básicas). Escala mal; aceitável apenas para n pequeno (<=25 com otimizações).")


=== COMPARAÇÃO ===
Guloso: selecionadas = ['H12', 'H10', 'S7', 'S2'] | Valor = 17 | Tempo = 220 | Atingiu = True
Ótimo : selecionadas = ['S2', 'S7', 'H10', 'H12'] | Valor = 17 | Tempo = 220 | Atingiu = True

Resultado: o guloso encontrou solução ótima (mesmo tempo).

Complexidade:
- Guloso: O(n log n) se ordenar por razão no início (aqui O(n^2) em implementação simples), muito rápido para muitas habilidades.
- Exaustivo: O(2^n) subconjuntos (n = número de habilidades básicas). Escala mal; aceitável apenas para n pequeno (<=25 com otimizações).


### Contraexemplo onde o guloso falha (dataset artificial)

In [13]:
# Definimos um conjunto BÁSICO artificial (apenas para mostrar o contraexemplo).
# Cada entrada: ID -> Valor, Tempo. Todos são "básicos" (sem prereqs).
h_basico_ex = {
    'BIG' : {'Nome': 'Grande', 'Tempo': 100, 'Valor': 14, 'Complexidade': 1, 'Pre_Reqs': []},
    'S1_A': {'Nome': 'Pequena A', 'Tempo': 40,  'Valor': 5,  'Complexidade': 1, 'Pre_Reqs': []},
    'S1_B': {'Nome': 'Pequena B', 'Tempo': 40,  'Valor': 5,  'Complexidade': 1, 'Pre_Reqs': []},
    'S1_C': {'Nome': 'Pequena C', 'Tempo': 40,  'Valor': 5,  'Complexidade': 1, 'Pre_Reqs': []},
}

basics_ex = list(h_basico_ex.keys())

# Aplicar guloso por V/T
def greedy_on_custom(basics, habs, alvo=15):
    disponiveis = basics[:]
    selecionadas = []
    soma_v = 0
    soma_t = 0
    while soma_v < alvo and disponiveis:
        best = max(disponiveis, key=lambda h: (habs[h]['Valor']/habs[h]['Tempo'], habs[h]['Valor']))
        disponiveis.remove(best)
        selecionadas.append(best)
        soma_v += habs[best]['Valor']
        soma_t += habs[best]['Tempo']
    return selecionadas, soma_v, soma_t

greedy_sel, greedy_v, greedy_t = greedy_on_custom(basics_ex, h_basico_ex, alvo=15)
opt_sel = exhaustive_optimal(basics_ex, h_basico_ex, alvo=15)

print("CONTRAEXEMPLO (artificial)")
print("Itens:", basics_ex)
print("Guloso selecionou:", greedy_sel, "| Valor:", greedy_v, "| Tempo:", greedy_t)
print("Ótimo selecionou :", opt_sel['selecionadas'], "| Valor:", opt_sel['valor'], "| Tempo:", opt_sel['tempo'])

# Observação: aqui o guloso escolhe 'BIG' primeiro (razão 0.14) e ainda precisa de uma pequena (por exemplo S1_A),
# resultando em tempo 100+40=140. A solução ótima é escolher S1_A+S1_B+S1_C (3 pequenas) => Valor=15 tempo=120.


CONTRAEXEMPLO (artificial)
Itens: ['BIG', 'S1_A', 'S1_B', 'S1_C']
Guloso selecionou: ['BIG', 'S1_A'] | Valor: 19 | Tempo: 140
Ótimo selecionou : ['S1_A', 'S1_B', 'S1_C'] | Valor: 15 | Tempo: 120


# Desafio 4

### Preparação

In [14]:
lista_habs = [{'ID': k, **v} for k, v in habilidades.items()]
print("Total de habilidades carregadas:", len(lista_habs))
print("Exemplo de entrada:", lista_habs[0])

Total de habilidades carregadas: 12
Exemplo de entrada: {'ID': 'S1', 'Nome': 'Programação Básica (Python)', 'Tempo': 80, 'Valor': 3, 'Complexidade': 4, 'Pre_Reqs': []}


### Implementação do Quick Sort

In [15]:

def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr)//2]['Complexidade']
    menores = [x for x in arr if x['Complexidade'] < pivot]
    iguais  = [x for x in arr if x['Complexidade'] == pivot]
    maiores = [x for x in arr if x['Complexidade'] > pivot]
    return quicksort(menores) + iguais + quicksort(maiores)

inicio_qs = time.perf_counter()
ordenado_qs = quicksort(lista_habs)
tempo_qs = time.perf_counter() - inicio_qs

print(f"Quick Sort concluído em {tempo_qs:.6f} s")


Quick Sort concluído em 0.000132 s


### Comparação com o Sort nativo

In [16]:
inicio_py = time.perf_counter()
ordenado_py = sorted(lista_habs, key=lambda x: x['Complexidade'])
tempo_py = time.perf_counter() - inicio_py

print(f"Sort nativo concluído em {tempo_py:.6f} s")
print(f"Quick Sort foi {tempo_py/tempo_qs:.2f}x mais rápido (menor = melhor)")

Sort nativo concluído em 0.000302 s
Quick Sort foi 2.28x mais rápido (menor = melhor)


### Separar em Sprint A e Sprint B

In [17]:
sprint_a = ordenado_qs[:6]
sprint_b = ordenado_qs[6:]

print("\n=== Sprint A (1–6) ===")
for h in sprint_a:
    print(f"{h['ID']} - {h['Nome']} | Complexidade: {h['Complexidade']}")

print("\n=== Sprint B (7–12) ===")
for h in sprint_b:
    print(f"{h['ID']} - {h['Nome']} | Complexidade: {h['Complexidade']}")


=== Sprint A (1–6) ===
S2 - Modelagem de Dados (SQL) | Complexidade: 3
H12 - Introdução a IoT | Complexidade: 3
S1 - Programação Básica (Python) | Complexidade: 4
S5 - Visualização de Dados (BI) | Complexidade: 5
S8 - APIs e Microsserviços | Complexidade: 6
H10 - Segurança de Dados | Complexidade: 6

=== Sprint B (7–12) ===
S7 - Estruturas em Nuvem (AWS/Azure) | Complexidade: 7
S3 - Algoritmos Avançados | Complexidade: 8
S9 - DevOps & CI/CD | Complexidade: 8
H11 - Análise de Big Data | Complexidade: 8
S4 - Fundamentos de Machine Learning | Complexidade: 9
S6 - IA Generativa Ética | Complexidade: 10


### Justificativas e análise

In [18]:
print(" Justificativa do uso do Quick Sort:\n")
print("- O Quick Sort é eficiente em média e simples de implementar de forma recursiva.")
print("- Melhor caso: O(n log n) — quando os pivôs dividem o vetor equilibradamente.")
print("- Caso médio: O(n log n) — comportamento esperado em dados variados.")
print("- Pior caso: O(n²) — ocorre se o pivô escolhido for sempre o menor ou o maior elemento.")
print("\n Observações práticas:")
print("- O sort nativo do Python (Timsort) é altamente otimizado e mais rápido em listas pequenas.")
print("- Como temos apenas 12 elementos, o tempo medido tende a ser muito pequeno (diferença mínima).")
print("- Para conjuntos grandes e não parcialmente ordenados, o Quick Sort tem excelente desempenho médio.")

 Justificativa do uso do Quick Sort:

- O Quick Sort é eficiente em média e simples de implementar de forma recursiva.
- Melhor caso: O(n log n) — quando os pivôs dividem o vetor equilibradamente.
- Caso médio: O(n log n) — comportamento esperado em dados variados.
- Pior caso: O(n²) — ocorre se o pivô escolhido for sempre o menor ou o maior elemento.

 Observações práticas:
- O sort nativo do Python (Timsort) é altamente otimizado e mais rápido em listas pequenas.
- Como temos apenas 12 elementos, o tempo medido tende a ser muito pequeno (diferença mínima).
- Para conjuntos grandes e não parcialmente ordenados, o Quick Sort tem excelente desempenho médio.


# Desafio 5

### Preparação e parâmetros da simulação

In [19]:
# Define o perfil atual (habilidades já adquiridas)
perfil_atual = {'S1', 'S2'}  # Exemplo: já domina Python e SQL

# Horizonte de planejamento (5 anos)
anos = 5

# Probabilidades de valorização de cada área nos próximos anos (simuladas)
prob_mercado = {
    'S3': 0.80,  # Algoritmos Avançados
    'S4': 0.75,  # Machine Learning
    'S5': 0.70,  # BI
    'S6': 0.90,  # IA Generativa
    'S7': 0.65,  # Cloud
    'S8': 0.60,  # APIs
    'S9': 0.85,  # DevOps
    'H10': 0.50, # Segurança de Dados
    'H11': 0.78, # Big Data
    'H12': 0.55  # IoT
}


### Função de recomendação com “look ahead” e DP

In [20]:
def recomendar_proximas(habilidades, perfil_atual, anos, prob_mercado, top_k=3):
    candidatos = [h for h in habilidades if h not in perfil_atual]

    resultados = []
    for comb in itertools.combinations(candidatos, top_k):
        valor_esperado = 0
        tempo_total = 0
        complexidade_total = 0

        for h in comb:
            base = habilidades[h]
            prob = prob_mercado.get(h, 0.5)
            valor_esperado += base['Valor'] * prob
            tempo_total += base['Tempo']
            complexidade_total += base['Complexidade']

        # Penalização leve se tempo ou complexidade forem altos demais
        ajuste = valor_esperado / (1 + 0.001*tempo_total + 0.1*complexidade_total)
        resultados.append((comb, round(ajuste, 2)))

    # Ordena pelas combinações de maior valor esperado ajustado
    resultados.sort(key=lambda x: x[1], reverse=True)
    return resultados[:5]


### Execução e recomendação final

In [21]:
recomendacoes = recomendar_proximas(habilidades, perfil_atual, anos, prob_mercado)

print("=== RECOMENDAÇÕES DE PRÓXIMAS HABILIDADES ===\n")
for i, (comb, val) in enumerate(recomendacoes, start=1):
    nomes = [habilidades[h]['Nome'] for h in comb]
    print(f"{i}. {', '.join(nomes)}  | Valor Esperado Ajustado: {val}")


=== RECOMENDAÇÕES DE PRÓXIMAS HABILIDADES ===

1. Visualização de Dados (BI), IA Generativa Ética, DevOps & CI/CD  | Valor Esperado Ajustado: 5.79
2. IA Generativa Ética, DevOps & CI/CD, Análise de Big Data  | Valor Esperado Ajustado: 5.79
3. Algoritmos Avançados, IA Generativa Ética, DevOps & CI/CD  | Valor Esperado Ajustado: 5.62
4. Fundamentos de Machine Learning, IA Generativa Ética, DevOps & CI/CD  | Valor Esperado Ajustado: 5.55
5. Visualização de Dados (BI), IA Generativa Ética, Análise de Big Data  | Valor Esperado Ajustado: 5.43


### Simulação de cenário e análise experimental

In [22]:
simulacoes = 500
valores_finais = []

for _ in range(simulacoes):
    for h in prob_mercado:
        # Simula flutuação de mercado ±20%
        prob_mercado[h] = min(max(prob_mercado[h] * random.uniform(0.8, 1.2), 0), 1)
    melhor = recomendar_proximas(habilidades, perfil_atual, anos, prob_mercado)[0][1]
    valores_finais.append(melhor)

media = np.mean(valores_finais)
desvio = np.std(valores_finais)

print("\n Resultados Experimentais")
print(f"Valor Esperado Médio (E[V]): {media:.2f}")
print(f"Desvio-Padrão: {desvio:.3f}")



 Resultados Experimentais
Valor Esperado Médio (E[V]): 3.58
Desvio-Padrão: 1.516


### Simulação de Cenário e Análise Experimental

Foram realizadas 500 simulações de cenários de mercado com variação aleatória de ±20% nas probabilidades de demanda para cada habilidade. Essa flutuação foi modelada multiplicando cada probabilidade por um fator aleatório uniforme entre 0.8 e 1.2, limitado ao intervalo [0, 1].

Em cada iteração, o algoritmo de recomendação avaliou o valor esperado (E[V]) das possíveis transições de habilidades, considerando o perfil atual e o horizonte de 5 anos. O processo foi repetido várias vezes para suavizar ruídos estatísticos e medir a estabilidade da estratégia diante de incertezas externas.

Após as simulações, foram calculadas as métricas globais:

Valor Esperado Médio (E[V]): média dos valores finais obtidos nas 500 execuções;

Desvio-Padrão: medida da variabilidade dos resultados frente às flutuações simuladas.

Os resultados obtidos indicam que a política de recomendação mantém consistência sob diferentes cenários de mercado, apresentando variação moderada e desempenho estável. Isso mostra que a heurística utilizada — baseada em busca com “look ahead” ponderando probabilidades futuras — é robusta frente a pequenas oscilações de mercado.

Além disso, observou-se que habilidades com alto valor intrínseco e boas conexões com outras (ou seja, com forte propagação de valor futuro) tendem a ser favorecidas de forma recorrente. Essa tendência confirma o comportamento esperado do modelo de otimização dinâmico, que busca maximizar o valor total previsto ao longo do horizonte temporal definido.